In [16]:
# Useful starting lines
%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from functions import *
from helpers import *
from proj1_helpers import *
from costs import *
from data_preprocessing import *

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Load the training data into feature matrix, class labels, and event ids:

In [17]:
from proj1_helpers import *
DATA_TRAIN_PATH = '../data/train.csv' 
# TODO: download train data and supply path here 
y, tX, ids = load_csv_data(DATA_TRAIN_PATH)
print("loading of the data : done")

loading of the data : done


## Data preprocessing

In [18]:
# reduction of the size of the training data set: 
# random selection of the data
num_samples=5000
seed=3
y, tX = select_random(y, tX, num_samples, seed)
print ("random selection of samples : done")
print( "number of samples : ", y.shape[0])

##### Splitting the data
xtrain1,xvalid1,ytrain,yvalid=split_data(tX, y, 0.7)
print( "splitting of the data : done ")
print("number of training samples :", xtrain1.shape[0])

#### Cleaning the data
#xtrain1bis=clean_data(xtrain1)
#xvalid1bis=clean_data(xvalid1)
#remove_threshold=0
#bad=clean_remove_data(xtrain1,remove_threshold)
#print("bad columns indices :", bad)
#xtrain1bis=np.delete(xtrain1,bad,axis=1)
#xvalid1bis=np.delete(xvalid1,bad,axis=1)

xtrain1bis = standardize(xtrain1)
xvalid1bis = standardize(xvalid1)

#### Data preprocessing : perform a pca on xtrain
ratio_pca=0.8

# finding the projector
U, k = my_pca(xtrain1bis, ratio_pca)
print("shape of the projector : ", U.shape)

# projecting the data 
xtrain2=np.dot(xtrain1bis,U)
xvalid2=np.dot(xvalid1bis,U)
print(" shape of the reduced training set : ", xtrain2.shape)
print(" shape of the reduced validation set : ", xvalid2.shape )

### Previous method
#then adding a one before each sample (to enable a constant in linear regression)
#xtrain=np.ones((xtrain2.shape[0],xtrain2.shape[1]+1))
#xvalid=np.ones((xvalid2.shape[0],xvalid2.shape[1]+1))
#xtrain[:,1:]=xtrain2
#xvalid[:,1:]=xvalid2

random selection of samples : done
number of samples :  5000
splitting of the data : done 
number of training samples : 3500
eigenvalues sorted by decreasing order :  [  4.34065467e+04   8.30372684e+03   7.92160943e+03   6.90823833e+03
   5.51932681e+03   5.40868934e+03   4.30343969e+03   4.12428925e+03
   3.53032114e+03   3.14607248e+03   2.67416358e+03   2.25657003e+03
   1.72002795e+03   1.58493693e+03   1.33527090e+03   9.08367256e+02
   5.25082098e+02   4.16773640e+02   3.22522419e+02   3.14074518e+02
   2.41699928e+02   1.25313007e+02   2.41877451e+00   4.39270344e-01
   2.64787068e-02   2.42072915e-02   1.73582851e-02   1.06860425e-02
   9.71418045e-04   4.68551034e-08]
 debugging : check the values :
k :  8
partial sum of eigenvalues :  85895.8663739
total sum of eigenvalues :  105000.0
ratio :  0.818055870228
shape of the projector :  (30, 8)
 shape of the reduced training set :  (3500, 8)
 shape of the reduced validation set :  (1500, 8)


## Machine Learning : test of the basic functions

In [36]:
### Training of the model : test of least_squares_GD
#weights = train_data(xtrain,ytrain,n_regression=1,gamma=0.0000001,max_iters=500)

### Training of the model : test of least_squares (analytical)
#weights = train_data(xtrain, ytrain, n_regression=3)

### Training of the model : test of least_squares_SGD
#weights = train_data(xtrain, ytrain, n_regression=2, gamma=10e-13, max_iters=500)

### Training of the model : test of ridge_regression
#weights = train_data(xtrain, ytrain, n_regression=4,lambd=100)

#print("Training of the model : done")

In [37]:
### Measure of the MSE for the trained model, over the validation set  : 

#MSE=compute_loss(yvalid,xvalid,weights) 

#print ( "MSE computed, value : ", MSE)

## Machine Learning :  Comparison of polynomial ridge regressions


In [39]:
# First comparison : for the given training and validation set : choice of the best lambd_ for the ridge regression
lambdas = np.logspace(-3, 3, 500) 
degree=7

#setting references
degree_ref=0
w_ref = 0
lambd_ref=0
score_ref = float("inf")
#loop over the degrees
for t in range(1,degree):
    
    #pre-processing the data according to the degree
    xtrain_rr=build_poly(xtrain2,t)
    xvalid_rr=build_poly(xvalid2,t)
    
    #loop over the lambdas
    for lambd_ in lambdas:
        
        #training the model for the ridge regression given lambda
        weights = train_data(xtrain_rr, ytrain, n_regression=4,lambd=lambd_)

        #computing its score
        #mse = compute_loss(yvalid, xvalid, weights)
        yvalid_pred=predict_labels(weights, xvalid_rr)
        score=0
        for i in range(yvalid.shape[0]):
            if (yvalid_pred[i]!=yvalid[i]):
                score=score+1

        #if its score is better than the previous one we keep the (weights, lambda couple)
        #if mse<loss_ref:
        if score<score_ref:
            w_ref=weights
            #loss_ref=mse
            score_ref=score
            lambd_ref=lambd_
            degree_ref=t
        # printing the result for this iteration    
        print("degree : ", str(t),  ", study of the parameter :", str(lambd_), " done. error associated : ", str(score_ref/yvalid.shape[0]) )



degree :  1 , study of the parameter : 0.001  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00102807322383  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00105693455356  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00108660611385  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.0011171106505  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00114847154784  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00118071284667  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00121385926269  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00124793620547  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.00128296979789  done. error associated :  0.2575
degree :  1 , study of the parameter : 0.0013189868962  done. error associated :  0.2575
degree :  1 , study of 

In [40]:
#saving the best parameters
degree_best_rr=degree_ref
weights_rr=w_ref
err_rr=score_ref/yvalid.shape[0]
#loss=loss_ref
print("comparison of the ridge regression parameters : done")
print("best parameter : " , str(lambd_ref))
print("best degree : ", str(degree_best_rr))
print(" associated error : " , str(err_rr))
print("associated weight vector" , str(weights_rr))

comparison of the ridge regression parameters : done
best parameter :  0.001
best degree :  5
 associated error :  0.22866666666666666
associated weight vector [  2.35161423e-01  -7.37255162e-02  -2.19526178e-02  -1.71496139e-03
  -1.22167909e-04  -6.46020866e-06  -4.93386294e-02   1.19831060e-02
   4.75724328e-03  -1.57938714e-03   1.07687219e-04   1.59238577e-01
   6.79495166e-03  -9.12231295e-03  -9.22819418e-04  -1.66814609e-05
   8.50631594e-02  -3.13925588e-02  -5.53305068e-03   7.53221638e-04
  -1.14734441e-05   2.73014994e-01  -4.41516070e-02  -9.85846814e-03
   2.59158818e-03  -1.34037781e-04  -1.38786705e-02  -4.34303918e-02
   1.19731325e-02  -6.46472840e-04  -2.13771537e-03   7.33424531e-03
  -5.68682239e-02  -3.50855113e-03   1.28068498e-02   5.68502941e-04
  -7.72956536e-02  -4.23204258e-02   8.76153442e-04   4.98606287e-04
   1.06304882e-05   2.93870789e-02   1.74105122e-02  -2.34199995e-02
  -8.56325812e-03   6.39222171e-03   1.82196649e-01   1.93329279e-02
  -1.0914187

## Training of the logistic regression

In [25]:
# data all ready preprocessed. 
# just need to add a column of ones
xtrain_log=np.ones((xtrain2.shape[0],xtrain2.shape[1]+1))
xtrain_log[:,1:]=xtrain2
xvalid_log=np.ones((xvalid2.shape[0],xvalid2.shape[1]+1))
xvalid_log[:,1:]=xvalid2

# preprocess the labels
ytrain_log = pre_process_logistic_training_labels(ytrain)

# perform a logistic regression on these labels 
loss, w_log = logistic_regression(ytrain_log, xtrain_log, gamma = 1e-7, max_iter = 50000, threshold = 1e-8)

# predict the labels of the validation set
yvalid_pred_log = predict_label_logistic_regression(xvalid_log, w_log)

# post process these predicted labels
yvalid_pred_log2 = post_process_logistic_predicted_labels(yvalid_pred_log)

# measure the error commited
err_log = measure_error(yvalid, yvalid_pred_log2)
print("error commited by the logistic regression : ", err_log)

Current iteration=0, the loss=[ 2426.01513196], the weight=[[ -5.50000000e-05]
 [ -1.01902686e-04]
 [ -6.02101310e-05]
 [ -4.19537245e-05]
 [  2.08488090e-05]
 [  2.80112322e-05]
 [ -5.67448133e-07]
 [  1.29774161e-07]
 [ -1.62321162e-05]]
Current iteration=100, the loss=[ 2406.93740448], the weight=[[ -5.53076643e-03]
 [ -9.75327671e-03]
 [ -6.01853771e-03]
 [ -4.19565337e-03]
 [  2.08764832e-03]
 [  2.80971646e-03]
 [ -5.69289370e-05]
 [  1.30379654e-05]
 [ -1.63102447e-03]]
Current iteration=200, the loss=[ 2389.98547194], the weight=[[ -1.09588247e-02]
 [ -1.84123792e-02]
 [ -1.18545076e-02]
 [ -8.26804619e-03]
 [  4.11904762e-03]
 [  5.55344618e-03]
 [ -1.12562628e-04]
 [  2.58205956e-05]
 [ -3.22930015e-03]]
Current iteration=300, the loss=[ 2374.79043552], the weight=[[ -1.63396006e-02]
 [ -2.61827724e-02]
 [ -1.75708038e-02]
 [ -1.22610414e-02]
 [  6.11564815e-03]
 [  8.26001469e-03]
 [ -1.67535874e-04]
 [  3.85046955e-05]
 [ -4.81132672e-03]]
Current iteration=400, the loss=[ 

## Selection of the best model

In [ ]:
# for now we just keep the parameters of ridge regression 
# later : comparison of the respective best errors on the validation set 

## Generate predictions and save ouput in csv format for submission:

In [41]:
DATA_TEST_PATH = '../data/test.csv'
_, tX_test, ids_test = load_csv_data(DATA_TEST_PATH)

In [42]:
# output path 
OUTPUT_PATH = '../results/result4.csv' 

# preprocess the submission data for the ridge regression: 
# 1) standardize
xtest = standardize(tX_test)
# 2) project 
xtest2 = np.dot(xtest, U)
# 3) build polynomial basis
xtest_rr =build_poly(xtest2, degree_best)

# preprocess the submission data for the logistic regression


In [43]:
# predict the labels and save the prediction in a csv file 

# for now : just predict according to the best ridge polynomial regression 
y_pred = predict_labels(weights_rr, xtest_rr)
create_csv_submission(ids_test, y_pred, OUTPUT_PATH)